In [ ]:
import sys
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper') 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from statistics import mean
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo

sns.set_theme(style="darkgrid")

# See complete data
pd.set_option('max_columns', None)
pd.set_option("max_rows", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()

In [ ]:
rename_col = {"MCP": "Price", "LimitPrice": "Price" , "Net_Cleared_Quantity": "Quantity", \
              "Broker's_Cleared_Quantity": "Quantity", "Broker's_Bidded_Quantity": "Quantity", \
              "Market_Position": "Quantity"}

In [ ]:
color_book = {'VV21': 'darkorchid', 'VidyutVanika': 'slateblue', 'VV20': 'slateblue', 'VidyutVanika18': 'yellowgreen', 'VV18': 'yellowgreen', \
              'Sample': 'orchid', 'AgentUDE': 'slategray', 'SPOT': 'orangered', 'TUC_TAC': 'gold', 'MCP': 'red'}

def get_colors(list_of_brokers):
    return [color_book.get(broker) for broker in list_of_brokers]

# Data Processing

In [ ]:
# Specify names of all the databases here
list_of_databases = ['PowerTAC2020_Wholesale_Test_VV21_2', 'PowerTAC2020_Wholesale_Test_VV18_2']

# Specify names of all the brokers corresponding to databases here
list_of_brokers = ['VV21', 'VV18']

# Specify list of the collections to be analysed here
# list_of_collections = helper_mongo.connect_to_mongo('PowerTAC2020_Wholesale_Test_VV21').list_collection_names()
list_of_collections = ['MarketTransaction_Info', 'Submitted_Bid_Information', 'ClearedTrade_Info']

# Specify the storage path along with directory
storage_path = '/mnt/d/PowerTAC/PowerTAC2021/experiments/wholesale_strategy_check/test'

# Keep all the collections at one place
dict_of_dataframes = dict()

In [ ]:
# View the Collections in all the databases

def view_collections(list_of_databases):
    
    for database in list_of_databases:
        
        db = helper_mongo.connect_to_mongo(database, server_ip='10.2.16.159', ssh_username='powertac', \
                                           ssh_password='sanjay9397', remote=True)
        print(database + "\n" + "-"*35)
        print(db.list_collection_names())
        print("\n")

# Function Calling
view_collections(list_of_databases)

In [ ]:
for dataset, broker in zip(list_of_databases, list_of_brokers):
    
    inner_dict = dict()
        
    for collection in list_of_collections:
        
        db = helper_mongo.query_to_mongo(dataset, collection, server_ip='10.2.16.159', ssh_username='powertac', \
                                         ssh_password='sanjay9397', remote=True)
        db.rename(columns = rename_col, inplace = True)
        
        db = db[abs(db['Quantity']) > 0.05] 
        # if (collection != 'ClearedTrade_Info'):
        #    # consider only bids
        #    db = db[db['Price'] < 0.0]
        
        if collection != 'Cash_and_Market_Position_Info':
            db['Proximity'] = db['Execution_Timeslot'] - db['Bidding_Timeslot']
            
        inner_dict.update({collection: db})
                
    dict_of_dataframes.update({broker: inner_dict})

In [ ]:
test_database = dict_of_dataframes.get('VV21').get('MarketTransaction_Info').copy(deep=True)
num_games = len(test_database.Game_Name.unique())

# Analysis and Plotting

### Limit-price and Clearing-price Comparison 

In [ ]:
###############################################
############ Plot using Plotly ################
###############################################
    
def limitprice_vs_mcp_plotly():    
    
    num_rows = 4
    num_cols = 1
    
    fig = make_subplots(rows=num_rows, 
                        cols=num_cols,
                        subplot_titles=('Overall Limit-price Comparison', 'Limit-price Comparsion (When Bids get Cleared)', 'Clearing-price Comparsion', 'MCP'),
                        vertical_spacing=0.10)
                    
    for broker, i in zip(list_of_brokers, range(1, len(list_of_brokers)+1)):
        
        db_sb = dict_of_dataframes.get(broker).get('Submitted_Bid_Information').copy(deep=True)
        db_mt = dict_of_dataframes.get(broker).get('MarketTransaction_Info').copy(deep=True)
        
        groupby_db_sb = (db_sb.groupby(['Game_Name', 'Execution_Timeslot']).mean()).groupby('Execution_Timeslot').mean()
        cp_sb = abs(groupby_db_sb['Price'])

        db = pd.merge(db_sb, db_mt,  how='right', 
                             left_on=['Game_Name', 'Bidding_Timeslot','Execution_Timeslot'], 
                             right_on = ['Game_Name', 'Bidding_Timeslot','Execution_Timeslot'])

        db = db[db['Price_y'] < 0.0] 
        groupby_db = (db.groupby(['Game_Name', 'Execution_Timeslot']).mean()).groupby('Execution_Timeslot').mean()
        lp = abs(groupby_db['Price_x'])
        cp = abs(groupby_db['Price_y'])    
    
        fig.append_trace(go.Scatter(x = cp_sb.index.tolist(),
                                    y = cp_sb, 
                                    name = broker,
                                    line=dict(color=color_book.get(broker)),
                                    legendgroup = 1
                                   ), row = 1, col = 1)
        
        fig.append_trace(go.Scatter(x = lp.index.tolist(),
                                    y = lp, 
                                    name = broker,
                                    line=dict(color=color_book.get(broker)),
                                    legendgroup = 1,
                                    showlegend=False
                                   ), row = 2, col = 1)
        
        fig.append_trace(go.Scatter(x = cp.index.tolist(),
                                    y = cp, 
                                    name = broker,
                                    line=dict(color=color_book.get(broker)),
                                    legendgroup = 2,
                                    showlegend=False
                                   ), row = 3, col = 1)
            
    db_ct = dict_of_dataframes.get('VV21').get('ClearedTrade_Info').copy(deep=True)
    groupby_db_ct = (db_ct.groupby(['Game_Name', 'Execution_Timeslot']).mean()).groupby('Execution_Timeslot').mean()
    cp_ct = groupby_db_ct['Price']
    
    fig.append_trace(go.Scatter(x = cp_ct.index.tolist(),
                                y = cp_ct, 
                                name = 'MCP',
                                line=dict(color=color_book.get('MCP')),
                                legendgroup = 1,
                               ), row = 4, col = 1)
    
    fig.update_yaxes(title_text="Price ($)")
    fig.update_xaxes(title_text="Timeslot", row = 4, col = 1)
    
    fig.update_layout(
        height=750, 
        width=1500, 
        template='ggplot2',
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    fig.write_html(storage_path + "/limitprice_vs_mcp_comparison.html")
    
###############################################
############### Function Call #################
###############################################    
    
limitprice_vs_mcp_plotly()

### Limit-price and MCP Heat-map 

In [ ]:
###############################################
############ Plot using Plotly ################
###############################################
    
def limitprice_vs_mcp_heatmap_plotly():    
    
    num_rows = 1
    num_cols = 1
    
    fig = make_subplots(rows=num_rows, 
                        cols=num_cols,
                        vertical_spacing=0.10)
                    
#     for broker, i in zip(list_of_brokers, range(1, len(list_of_brokers)+1)):
        
#         db_sb = dict_of_dataframes.get(broker).get('Submitted_Bid_Information').copy(deep=True)
#         db_mt = dict_of_dataframes.get(broker).get('MarketTransaction_Info').copy(deep=True)
        
#         groupby_db_sb = (db_sb.groupby(['Game_Name', 'Execution_Timeslot']).mean()).groupby('Execution_Timeslot').mean()
#         cp_sb = abs(groupby_db_sb['Price'])

#         db = pd.merge(db_sb, db_mt,  how='right', 
#                              left_on=['Game_Name', 'Bidding_Timeslot','Execution_Timeslot'], 
#                              right_on = ['Game_Name', 'Bidding_Timeslot','Execution_Timeslot'])

#         db = db[db['Price_y'] < 0.0] 
#         groupby_db = (db.groupby(['Game_Name', 'Execution_Timeslot']).mean()).groupby('Execution_Timeslot').mean()
#         lp = abs(groupby_db['Price_x'])
#         cp = abs(groupby_db['Price_y'])    
    
#         fig.append_trace(go.Scatter(x = cp_sb.index.tolist(),
#                                     y = cp_sb, 
#                                     name = broker,
#                                     line=dict(color=color_book.get(broker)),
#                                     legendgroup = 1
#                                    ), row = 1, col = 1)
        
#         fig.append_trace(go.Scatter(x = lp.index.tolist(),
#                                     y = lp, 
#                                     name = broker,
#                                     line=dict(color=color_book.get(broker)),
#                                     legendgroup = 1,
#                                     showlegend=False
#                                    ), row = 2, col = 1)
        
#         fig.append_trace(go.Scatter(x = cp.index.tolist(),
#                                     y = cp, 
#                                     name = broker,
#                                     line=dict(color=color_book.get(broker)),
#                                     legendgroup = 2,
#                                     showlegend=False
#                                    ), row = 3, col = 1)
            
    db_ct = dict_of_dataframes.get('VV21').get('ClearedTrade_Info').copy(deep=True)
    groupby_db_ct = (db_ct.groupby(['Game_Name', 'Execution_Timeslot']).mean()).groupby('Execution_Timeslot').mean()
    cp_ct = groupby_db_ct['Price']
    
    fig.append_trace(go.Scatter(x = cp_ct.index.tolist(),
                                y = cp_ct, 
                                name = 'MCP',
                                line=dict(color=color_book.get('MCP')),
                                legendgroup = 1,
                               ), row = 4, col = 1)
    
    fig.update_yaxes(title_text="Price ($)")
    fig.update_xaxes(title_text="Timeslot", row = 4, col = 1)
    
    fig.update_layout(
        height=750, 
        width=1500, 
        template='ggplot2',
    )
    
    for i in fig['layout']['annotations']:
        i['font'] = dict(size=14)
    
    fig.write_html(storage_path + "/limitprice_vs_mcp_heatmap.html")
    
###############################################
############### Function Call #################
###############################################    
    
limitprice_vs_mcp_heatmap_plotly()

In [ ]:
db_ct = dict_of_dataframes.get('VV21').get('ClearedTrade_Info').copy(deep=True)
db_ct = db_ct[db_ct['Game_Name'] == 'finals_2019_07_10_2947'][['Proximity', 'Execution_Timeslot', 'Price', 'Quantity']]
out = db_ct.groupby('Execution_Timeslot').Price.apply(list).reset_index()
out = np.array(out['Price'][24:-24])

In [ ]:
b_out = np.zeros((24,1))

for i in out:
    a = np.array(i).reshape(-1,1)
    if a.shape[0] == 24:
        b_out = np.append(b_out, a, axis=1)
    
b_out.shape